In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv' , index=False)
df.head()

,review,sentiment
162,"I got a chance to talk with the co-creator, Re...",positive
517,An example of genius filmaking. The epic story...,positive
555,The name (Frau) of the main character is the G...,negative
580,Blonde and Blonder has Pamela Anderson and Den...,positive
703,"Bill Rebane's ""The Capture of Bigfoot"" is one ...",negative


In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [12]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [13]:
df = normalize_text(df)
df.head()

,review,sentiment
162,got chance talk co creator rebecca cammisa hig...,positive
517,example genius filmaking epic story three majo...,positive
555,name frau main character german word woman kno...,negative
580,blonde blonder pamela anderson denise richards...,positive
703,bill rebane capture bigfoot one awful horror m...,negative


In [14]:
df['sentiment'].value_counts()

sentiment
positive    251
negative    249
Name: count, dtype: int64

In [15]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [16]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
162,got chance talk co creator rebecca cammisa hig...,1
517,example genius filmaking epic story three majo...,1
555,name frau main character german word woman kno...,0
580,blonde blonder pamela anderson denise richards...,1
703,bill rebane capture bigfoot one awful horror m...,0


In [17]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [26]:
vectorizer = CountVectorizer(max_features=150)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [28]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/Rathodpradip1/Capstone-Project.mlflow")
dagshub.init(repo_owner='Rathodpradip1', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

2025-08-12 11:45:18,749 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Rathodpradip1/Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Rathodpradip1/Capstone-Project"

2025-08-12 11:45:18,762 - INFO - Initialized MLflow to track repo "Rathodpradip1/Capstone-Project"


Repository Rathodpradip1/Capstone-Project initialized!

2025-08-12 11:45:18,765 - INFO - Repository Rathodpradip1/Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/a9e3c23265c2415288597d60fe003ec7', creation_time=1754977972025, experiment_id='0', last_update_time=1754977972025, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [29]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 150)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-08-12 11:45:26,178 - INFO - Starting MLflow run...
2025-08-12 11:45:26,663 - INFO - Logging preprocessing parameters...
2025-08-12 11:45:27,765 - INFO - Initializing Logistic Regression model...
2025-08-12 11:45:27,765 - INFO - Fitting the model...
2025-08-12 11:45:27,819 - INFO - Model training complete.
2025-08-12 11:45:27,820 - INFO - Logging model parameters...
2025-08-12 11:45:28,210 - INFO - Making predictions...
2025-08-12 11:45:28,210 - INFO - Calculating evaluation metrics...
2025-08-12 11:45:28,230 - INFO - Logging evaluation metrics...
2025-08-12 11:45:32,708 - INFO - Saving and logging the model...
2025/08/12 11:45:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-08-12 11:45:34,530 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_3480\3555150425.py", line 48, in <mod

🏃 View run flawless-cow-219 at: https://dagshub.com/Rathodpradip1/Capstone-Project.mlflow/#/experiments/0/runs/afa4a3ed6c984c2382b457792cba2a47
🧪 View experiment at: https://dagshub.com/Rathodpradip1/Capstone-Project.mlflow/#/experiments/0
